In [7]:
import requests
import pandas as pd

df = pd.read_excel('output/müslüm_gürses_tr_20251203_131634.xlsx')

print(df['Yorum'].head(20))
print("------------------------------------------------------------------------------")
print(df["Yorum"].tail(20))


0     00:00 ●  Vefasız Alem - Söz: Ali Tekintüre - M...
1                                                  4:30
2                                                     😂
3                                                  5:17
4                                                     ❤
5     Gerçekten öyle bir yudum mutluluğa bir ömür is...
6                                           müslüm baba
7                                     Genclik yillarimi
8                                     😥😔❤️‍🩹❤️‍🩹❤️‍🩹👑👏👏
9                            👑❤️❤️❤️❤️❤️❤️🌹🌹🌹🌹👑🙏🙏👑🙏🙏🌹🌹🌹
10          Senin gibisi yok be baba arabeskin kralisin
11                       ❤❤❤❤❤❤❤❤❤❤❤❤❤❤❤❤❤❤❤❤❤😢😢😢😢😢😢😢😢😢
12    Saygı değer esen müzik ekibi emeklerinize sons...
13                                                41:00
14    GELMEZ BİR DAHA BABA GİBİSİ MEKANIN CENNET TOM...
15                                  damarsa baba müslüm
16                                       Love you man ❤
17    Haklisin baba dertsiz olan bir insan gece 

In [8]:
from timeit import default_timer as timer
from datetime import timedelta
import re
from nltk.corpus import stopwords

In [9]:
try:
    stop_kelimeler = set(stopwords.words('turkish'))
except LookupError:
    import nltk
    nltk.download('stopwords')
    stop_kelimeler = set(stopwords.words('turkish'))

In [10]:
def preprocessing(series, remove_hashtag=False, remove_mentions=False, remove_links=False, 
                  remove_numbers=False, remove_short_text=False, lowercase=False, 
                  remove_punctuation=False, remove_stopwords=False, remove_rare_words=False,remove_non_latin=False,
                  rare_limit=5, custom_stopwords=None, min_text_length=3):
    """
    Preprocess text data in a pandas Series.

    Parameters
    ----------
    series : pandas.Series
        The series to be processed.
    remove_hashtag : bool, default=False
        If True, removes hashtags (#).
    remove_mentions : bool, default=False
        If True, removes mentions (@).
    remove_links : bool, default=False
        If True, removes URLs.
    remove_numbers : bool, default=False
        If True, removes numeric characters.
    remove_short_text : bool, default=False
        If True, removes short words (length < min_text_length).
    lowercase : bool, default=False
        If True, converts all text to lowercase.
    remove_punctuation : bool, default=False
        If True, removes punctuation marks.
    remove_stopwords : bool, default=False
        If True, removes Turkish stopwords.
    remove_rare_words : bool, default=False
        If True, removes rare words (frequency <= rare_limit).
    rare_limit : int, default=5
        Frequency threshold for rare word removal.
    custom_stopwords : set or None, default=None
        Custom stopwords to use instead of default Turkish stopwords.
    min_text_length : int, default=3
        Minimum word length when remove_short_text=True.

    Returns
    -------
    pandas.Series
        The preprocessed series.
    """
    # Create a copy to avoid modifying original
    series = series.copy()
    
    if lowercase:
        print("Converting to lowercase...")
        start = timer()
        series = series.str.lower()
        print(f"Completed in {timedelta(seconds=timer() - start)}")

    if remove_hashtag:
        print("Removing hashtags...")
        start = timer()
        series = series.str.replace(r'#\w+', '', regex=True)
        print(f"Completed in {timedelta(seconds=timer() - start)}")

    if remove_mentions:
        print("Removing mentions...")
        start = timer()
        series = series.str.replace(r'@\w+', '', regex=True)
        print(f"Completed in {timedelta(seconds=timer() - start)}")

    if remove_links:
        print("Removing links...")
        start = timer()
        series = series.str.replace(r'http\S+|www\.\S+', '', regex=True)
        print(f"Completed in {timedelta(seconds=timer() - start)}")

    if remove_numbers:
        print("Removing numbers...")
        start = timer()
        series = series.str.replace(r'\d+', '', regex=True)
        print(f"Completed in {timedelta(seconds=timer() - start)}")

    if remove_punctuation:
        print("Removing punctuation...")
        start = timer()
        series = series.str.replace(r'[^\w\s]', '', regex=True)
        print(f"Completed in {timedelta(seconds=timer() - start)}")

    if remove_short_text:
        print(f"Removing short words (length < {min_text_length})...")
        start = timer()
        pattern = r'\b\w{1,' + str(min_text_length - 1) + r'}\b'
        series = series.str.replace(pattern, '', regex=True)
        print(f"Completed in {timedelta(seconds=timer() - start)}")

    if remove_stopwords:
        print("Removing stopwords...")
        start = timer()
        stopwords_to_use = custom_stopwords if custom_stopwords else stop_kelimeler
        series = series.apply(lambda x: ' '.join([word for word in str(x).split() 
                                                   if word.lower() not in stopwords_to_use]))
        print(f"Completed in {timedelta(seconds=timer() - start)}")
    
    if remove_non_latin:
        print("Removing non-latin characters (Arabic, Cyrillic, Emojis)...")
        start = timer()
        # Mantık: a-z, 0-9 ve Türkçe harfler (çğıöşü) HARİÇ her şeyi sil.
        # ^ işareti "haricinde" demektir.
        series = series.str.replace(r'[^a-zA-Z0-9çğıöşüÇĞİÖŞÜ\s]', '', regex=True)
        print(f"Completed in {timedelta(seconds=timer() - start)}")

    if remove_rare_words:
        print("Removing rare words...")
        start = timer()
        all_words = ' '.join(series.astype(str)).split()
        word_counts = pd.Series(all_words).value_counts()
        
        print(f"Total unique words: {len(word_counts)}")
        rare_words = word_counts[word_counts <= rare_limit]
        print(f"Rare words ({rare_limit} or fewer occurrences): {len(rare_words)} "
              f"({len(rare_words)/len(word_counts)*100:.2f}%)")
        
        rare_words_set = set(rare_words.index)
        series = series.apply(lambda x: ' '.join([word for word in str(x).split() 
                                                   if word not in rare_words_set]))
        print(f"Completed in {timedelta(seconds=timer() - start)}")

    # Clean up extra whitespace
    series = series.str.strip().str.replace(r'\s+', ' ', regex=True)
    
    return series

In [13]:
def tr_en_char_translate(series):
    """
    Convert Turkish characters to English characters.

    Parameters
    ----------
    series : pandas.Series
        The series to be translated.

    Returns
    -------
    pandas.Series
        The translated series.
    """
    series = series.str.replace('ı', 'i')
    series = series.str.replace('ü', 'u')
    series = series.str.replace('ö', 'o')
    series = series.str.replace('ğ', 'g')
    series = series.str.replace('ş', 's')
    series = series.str.replace('ç', 'c')
    return series

In [12]:
df['Yorum_lowercased'] = preprocessing(df['Yorum'], lowercase=True)
df[['Yorum', 'Yorum_lowercased']].head(20)

Converting to lowercase...
Completed in 0:00:00.056354


,Yorum,Yorum_lowercased
0,00:00 ● Vefasız Alem - Söz: Ali Tekintüre - M...,00:00 ● vefasız alem - söz: ali tekintüre - mü...
1,4:30,4:30
2,😂,😂
3,5:17,5:17
4,❤,❤
5,Gerçekten öyle bir yudum mutluluğa bir ömür is...,gerçekten öyle bir yudum mutluluğa bir ömür is...
6,müslüm baba,müslüm baba
7,Genclik yillarimi,genclik yillarimi
8,😥😔❤️‍🩹❤️‍🩹❤️‍🩹👑👏👏,😥😔❤️‍🩹❤️‍🩹❤️‍🩹👑👏👏
9,👑❤️❤️❤️❤️❤️❤️🌹🌹🌹🌹👑🙏🙏👑🙏🙏🌹🌹🌹,👑❤️❤️❤️❤️❤️❤️🌹🌹🌹🌹👑🙏🙏👑🙏🙏🌹🌹🌹


In [14]:
df['Yorum_mentions'] = preprocessing(df['Yorum_lowercased'], remove_mentions=True)
df[['Yorum_lowercased', 'Yorum_mentions']].head(20)

Removing mentions...
Completed in 0:00:00.037869


,Yorum_lowercased,Yorum_mentions
0,00:00 ● vefasız alem - söz: ali tekintüre - mü...,00:00 ● vefasız alem - söz: ali tekintüre - mü...
1,4:30,4:30
2,😂,😂
3,5:17,5:17
4,❤,❤
5,gerçekten öyle bir yudum mutluluğa bir ömür is...,gerçekten öyle bir yudum mutluluğa bir ömür is...
6,müslüm baba,müslüm baba
7,genclik yillarimi,genclik yillarimi
8,😥😔❤️‍🩹❤️‍🩹❤️‍🩹👑👏👏,😥😔❤️‍🩹❤️‍🩹❤️‍🩹👑👏👏
9,👑❤️❤️❤️❤️❤️❤️🌹🌹🌹🌹👑🙏🙏👑🙏🙏🌹🌹🌹,👑❤️❤️❤️❤️❤️❤️🌹🌹🌹🌹👑🙏🙏👑🙏🙏🌹🌹🌹


In [15]:
df['Yorum_links'] = preprocessing(df['Yorum_mentions'], remove_links=True)
df[['Yorum_mentions', 'Yorum_links']].head(20)

Removing links...
Completed in 0:00:00.075014


,Yorum_mentions,Yorum_links
0,00:00 ● vefasız alem - söz: ali tekintüre - mü...,00:00 ● vefasız alem - söz: ali tekintüre - mü...
1,4:30,4:30
2,😂,😂
3,5:17,5:17
4,❤,❤
5,gerçekten öyle bir yudum mutluluğa bir ömür is...,gerçekten öyle bir yudum mutluluğa bir ömür is...
6,müslüm baba,müslüm baba
7,genclik yillarimi,genclik yillarimi
8,😥😔❤️‍🩹❤️‍🩹❤️‍🩹👑👏👏,😥😔❤️‍🩹❤️‍🩹❤️‍🩹👑👏👏
9,👑❤️❤️❤️❤️❤️❤️🌹🌹🌹🌹👑🙏🙏👑🙏🙏🌹🌹🌹,👑❤️❤️❤️❤️❤️❤️🌹🌹🌹🌹👑🙏🙏👑🙏🙏🌹🌹🌹


In [16]:
df['Yorum_numbers'] = preprocessing(df['Yorum_links'], remove_numbers=True)
df[['Yorum_links', 'Yorum_numbers']].head(20)

Removing numbers...
Completed in 0:00:00.138173


,Yorum_links,Yorum_numbers
0,00:00 ● vefasız alem - söz: ali tekintüre - mü...,: ● vefasız alem - söz: ali tekintüre - müzik:...
1,4:30,:
2,😂,😂
3,5:17,:
4,❤,❤
5,gerçekten öyle bir yudum mutluluğa bir ömür is...,gerçekten öyle bir yudum mutluluğa bir ömür is...
6,müslüm baba,müslüm baba
7,genclik yillarimi,genclik yillarimi
8,😥😔❤️‍🩹❤️‍🩹❤️‍🩹👑👏👏,😥😔❤️‍🩹❤️‍🩹❤️‍🩹👑👏👏
9,👑❤️❤️❤️❤️❤️❤️🌹🌹🌹🌹👑🙏🙏👑🙏🙏🌹🌹🌹,👑❤️❤️❤️❤️❤️❤️🌹🌹🌹🌹👑🙏🙏👑🙏🙏🌹🌹🌹


In [17]:
df['Yorum_short_text'] = preprocessing(df['Yorum_numbers'], remove_short_text=True)
df[['Yorum_numbers', 'Yorum_short_text']].head(20)

Removing short words (length < 3)...
Completed in 0:00:00.274905


,Yorum_numbers,Yorum_short_text
0,: ● vefasız alem - söz: ali tekintüre - müzik:...,: ● vefasız alem - söz: ali tekintüre - müzik:...
1,:,:
2,😂,😂
3,:,:
4,❤,❤
5,gerçekten öyle bir yudum mutluluğa bir ömür is...,gerçekten öyle bir yudum mutluluğa bir ömür is...
6,müslüm baba,müslüm baba
7,genclik yillarimi,genclik yillarimi
8,😥😔❤️‍🩹❤️‍🩹❤️‍🩹👑👏👏,😥😔❤️‍🩹❤️‍🩹❤️‍🩹👑👏👏
9,👑❤️❤️❤️❤️❤️❤️🌹🌹🌹🌹👑🙏🙏👑🙏🙏🌹🌹🌹,👑❤️❤️❤️❤️❤️❤️🌹🌹🌹🌹👑🙏🙏👑🙏🙏🌹🌹🌹


In [18]:
df['Yorum_punctuation'] = preprocessing(df['Yorum_short_text'], remove_punctuation=True)
df[['Yorum_short_text', 'Yorum_punctuation']].head(20)

Removing punctuation...
Completed in 0:00:00.186876


,Yorum_short_text,Yorum_punctuation
0,: ● vefasız alem - söz: ali tekintüre - müzik:...,vefasız alem söz ali tekintüre müzik atilla al...
1,:,
2,😂,
3,:,
4,❤,
5,gerçekten öyle bir yudum mutluluğa bir ömür is...,gerçekten öyle bir yudum mutluluğa bir ömür is...
6,müslüm baba,müslüm baba
7,genclik yillarimi,genclik yillarimi
8,😥😔❤️‍🩹❤️‍🩹❤️‍🩹👑👏👏,
9,👑❤️❤️❤️❤️❤️❤️🌹🌹🌹🌹👑🙏🙏👑🙏🙏🌹🌹🌹,


In [19]:
df['Yorum_rare_words'] = preprocessing(df['Yorum_punctuation'], remove_rare_words=True, rare_limit=1)
df[['Yorum_punctuation', 'Yorum_rare_words']].head(20)

Removing rare words...
Total unique words: 135015
Rare words (1 or fewer occurrences): 87227 (64.61%)
Completed in 0:00:00.527906


,Yorum_punctuation,Yorum_rare_words
0,vefasız alem söz ali tekintüre müzik atilla al...,vefasız alem söz ali tekintüre müzik atilla al...
1,,
2,,
3,,
4,,
5,gerçekten öyle bir yudum mutluluğa bir ömür is...,gerçekten öyle bir yudum mutluluğa bir ömür is...
6,müslüm baba,müslüm baba
7,genclik yillarimi,genclik yillarimi
8,,
9,,


In [20]:
df['Yorum_stopwords'] = preprocessing(df['Yorum_rare_words'], remove_stopwords=True)
df[['Yorum_rare_words', 'Yorum_stopwords']].head(30)

Removing stopwords...
Completed in 0:00:00.221350


,Yorum_rare_words,Yorum_stopwords
0,vefasız alem söz ali tekintüre müzik atilla al...,vefasız alem söz ali tekintüre müzik atilla al...
1,,
2,,
3,,
4,,
5,gerçekten öyle bir yudum mutluluğa bir ömür is...,gerçekten öyle bir yudum mutluluğa bir ömür is...
6,müslüm baba,müslüm baba
7,genclik yillarimi,genclik yillarimi
8,,
9,,


In [21]:
df['Yorum_non_latin'] = preprocessing(df['Yorum_rare_words'], remove_non_latin=True) #Türkçe ve ingilzice harfler dışındaki karakterleri kaldırdırdım.
df[['Yorum_rare_words', 'Yorum_non_latin']].head(30)

Removing non-latin characters (Arabic, Cyrillic, Emojis)...
Completed in 0:00:00.076623


,Yorum_rare_words,Yorum_non_latin
0,vefasız alem söz ali tekintüre müzik atilla al...,vefasız alem söz ali tekintüre müzik atilla al...
1,,
2,,
3,,
4,,
5,gerçekten öyle bir yudum mutluluğa bir ömür is...,gerçekten öyle bir yudum mutluluğa bir ömür is...
6,müslüm baba,müslüm baba
7,genclik yillarimi,genclik yillarimi
8,,
9,,


In [22]:
df['Yorum_tr_en_char'] = tr_en_char_translate(df['Yorum_non_latin'])
df[['Yorum_non_latin', 'Yorum_tr_en_char']].head(30)

,Yorum_non_latin,Yorum_tr_en_char
0,vefasız alem söz ali tekintüre müzik atilla al...,vefasiz alem soz ali tekinture muzik atilla al...
1,,
2,,
3,,
4,,
5,gerçekten öyle bir yudum mutluluğa bir ömür is...,gercekten oyle bir yudum mutluluga bir omur is...
6,müslüm baba,muslum baba
7,genclik yillarimi,genclik yillarimi
8,,
9,,


In [23]:
import numpy as np

df['Yorum_tr_en_char_last'] = df['Yorum_tr_en_char']

df['Yorum_tr_en_char_last'] = df['Yorum_tr_en_char_last'].replace(r'^\s*$', np.nan, regex=True)


df.dropna(subset=['Yorum_tr_en_char_last'], inplace=True) #içinde boş olan satırları attık.


df.reset_index(drop=True, inplace=True) #indeksi düzeltmek için yaptık çünkü boş satırları attık veri setinden.

# Sonucu Yazdır
print(f"Temizlik bitti! Kalan toplam dolu satır sayısı: {len(df)}")
df[['Yorum_tr_en_char', 'Yorum_tr_en_char_last']].head(30) 

Temizlik bitti! Kalan toplam dolu satır sayısı: 129780


,Yorum_tr_en_char,Yorum_tr_en_char_last
0,vefasiz alem soz ali tekinture muzik atilla al...,vefasiz alem soz ali tekinture muzik atilla al...
1,gercekten oyle bir yudum mutluluga bir omur is...,gercekten oyle bir yudum mutluluga bir omur is...
2,muslum baba,muslum baba
3,genclik yillarimi,genclik yillarimi
4,senin gibisi yok baba arabeskin kralisin,senin gibisi yok baba arabeskin kralisin
5,saygi deger esen muzik ekibi sonsuz album nisan,saygi deger esen muzik ekibi sonsuz album nisan
6,gelmez daha baba mekanin cennet mem olsun baba,gelmez daha baba mekanin cennet mem olsun baba
7,damarsa baba muslum,damarsa baba muslum
8,love you man,love you man
9,haklisin baba dertsiz olan bir insan gece gunduz,haklisin baba dertsiz olan bir insan gece gunduz


In [24]:
from collections import Counter
import re

def preprocess_text(text):
    words = text.split()  
    return words


def analyze_columns(df, column1, column2):
    
    print("Sütunlardaki metinler birleştiriliyor.")
    words1 = df[column1].fillna('').astype(str).apply(preprocess_text).sum()
    words2 = df[column2].fillna('').astype(str).apply(preprocess_text).sum()

    
    print("Toplam kelime sayıları hesaplanıyor.")
    total_words1 = len(words1)
    total_words2 = len(words2)

   
    print("Farklı kelime sayıları hesaplanıyor.")
    unique_words1 = len(set(words1))
    unique_words2 = len(set(words2))

    
    print("En çok tekrar eden 10 kelime hesaplanıyor.")
    common_words1 = Counter(words1).most_common(10)
    common_words2 = Counter(words2).most_common(10)

    
    print("En az tekrar eden 10 kelime hesaplanıyor.")
    least_common_words1 = Counter(words1).most_common()[:-11:-1]
    least_common_words2 = Counter(words2).most_common()[:-11:-1]

   
    print(f"Sütun 1: {column1}")
    print(f"Toplam kelime sayısı: {total_words1}")
    print(f"Farklı kelime sayısı: {unique_words1}")
    print(f"En çok tekrar eden 10 kelime: {common_words1}")
    print(f"En az tekrar eden 10 kelime: {least_common_words1}")
    print("\n")
    print(f"Sütun 2: {column2}")
    print(f"Toplam kelime sayısı: {total_words2}")
    print(f"Farklı kelime sayısı: {unique_words2}")
    print(f"En çok tekrar eden 10 kelime: {common_words2}")
    print(f"En az tekrar eden 10 kelime: {least_common_words2}")


# DataFrame'deki sütunları analiz et
analyze_columns(df, 'Yorum', 'Yorum_tr_en_char_last')


Sütunlardaki metinler birleştiriliyor.
Toplam kelime sayıları hesaplanıyor.
Farklı kelime sayıları hesaplanıyor.
En çok tekrar eden 10 kelime hesaplanıyor.
En az tekrar eden 10 kelime hesaplanıyor.
Sütun 1: Yorum
Toplam kelime sayısı: 1377324
Farklı kelime sayısı: 211080
En çok tekrar eden 10 kelime: [('bir', 18227), ('bu', 15902), ('baba', 13864), ('Müslüm', 9253), ('olsun', 8825), ('çok', 8588), ('ne', 7981), ('ve', 7371), ('ama', 7202), ('seni', 6958)]
En az tekrar eden 10 kelime: [('omit', 1), ('coşgun', 1), ('soyleyebılırbu', 1), ('Keleş', 1), ('zararsiz', 1), ('yatsin,ne', 1), ('Kabrinde', 1), ('sarkiyin!', 1), ('gormemisiz', 1), ('Unutmayalım...', 1)]


Sütun 2: Yorum_tr_en_char_last
Toplam kelime sayısı: 1123808
Farklı kelime sayısı: 41102
En çok tekrar eden 10 kelime: [('baba', 23403), ('bir', 21345), ('muslum', 18474), ('cok', 12182), ('olsun', 11172), ('seni', 9940), ('ama', 8375), ('ben', 8226), ('allah', 6937), ('sen', 6618)]
En az tekrar eden 10 kelime: [('gunlunce', 2), 

In [17]:
df.columns

Index(['Yorum', 'Yorum_lowercased', 'Yorum_mentions', 'Yorum_links',
       'Yorum_numbers', 'Yorum_short_text', 'Yorum_punctuation',
       'Yorum_rare_words', 'Yorum_stopwords', 'Yorum_non_latin',
       'Yorum_tr_en_char', 'Yorum_tr_en_char_last'],
      dtype='object')

In [18]:

silinecek_sutunlar = [
      'Yorum_lowercased', 'Yorum_mentions', 'Yorum_links',
       'Yorum_numbers', 'Yorum_short_text', 'Yorum_punctuation',
       'Yorum_rare_words', 'Yorum_stopwords', 'Yorum_non_latin',
       'Yorum_tr_en_char',
]

df.drop(silinecek_sutunlar, axis=1, inplace=True, errors='ignore')

print("Ara sütunlar silindi. Kalan sütunlar:")
print(df.columns.tolist())
df.head()

Ara sütunlar silindi. Kalan sütunlar:
['Yorum', 'Yorum_tr_en_char_last']


,Yorum,Yorum_tr_en_char_last
0,00:00 ● Vefasız Alem - Söz: Ali Tekintüre - M...,vefasiz alem soz ali tekinture muzik atilla al...
1,Gerçekten öyle bir yudum mutluluğa bir ömür is...,gercekten oyle bir yudum mutluluga bir omur is...
2,müslüm baba,muslum baba
3,Genclik yillarimi,genclik yillarimi
4,Senin gibisi yok be baba arabeskin kralisin,senin gibisi yok baba arabeskin kralisin


In [19]:
df.tail(30)

,Yorum,Yorum_tr_en_char_last
129750,baba buyuksun,baba buyuksun
129751,Solmadan gel artık aşkımın gülü dedik ama gelm...,solmadan gel artik askimin gulu dedik ama baba...
129752,Tarık Demirci bence en güzel şarkısı HERŞEY GÜ...,tarik demirci bence guzel sarkisi hersey gunlu...
129753,RONYA Korkmaz tertemiz aşkların gönüllü spons...,korkmaz tertemiz asklarin gonullu sponsoru mus...
129754,Tarık Demirci evet gerçekten çok dokunaklı öze...,tarik demirci evet gercekten cok dokunakli muzigi
129755,Kalp sensiz olmaz baba,kalp sensiz olmaz baba
129756,Baba,baba
129757,albüm değil sanki şampiyonlar ligi Allah gani ...,album degil sanki sampiyonlar ligi allah gani ...
129758,İbrahim Taşkın o kadarrrrrrr 👍😎,brahim taskin
129759,Baba'nın en az 100 şarkısı hit olur....ama küs...,babanin sarkisi hit olurama kuskunum albumu ad...


In [20]:

dosya_ismi = "muslum_gurses_yorumlari_temizlenmis.xlsx"

df.to_excel(dosya_ismi, index=False)

print(f"Harika! Dosya '{dosya_ismi}' adıyla başarıyla kaydedildi.")

Harika! Dosya 'muslum_gurses_yorumlari_temizlenmis.xlsx' adıyla başarıyla kaydedildi.


In [1]:
import pandas as pd

df_yeni = pd.read_excel("muslum_gurses_yorumlari_temizlenmis.xlsx")
df_yeni.columns

Index(['Yorum', 'Yorum_tr_en_char_last'], dtype='object')

In [2]:
df_yeni.rename(columns={'Yorum_tr_en_char_last': 'Temizlenmis_Yorumlar'}, inplace=True)

In [3]:
df_yeni.to_excel("muslum_gurses_yorumlari_temizlenmis.xlsx", index=False)

In [5]:
df_yeni.columns

Index(['Yorum', 'Temizlenmis_Yorumlar'], dtype='object')